In [1]:
# matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
# pandas
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.3f' % x)
# numpy
import numpy as np
# keras; not sure which version
# API changes frequently, so...
import keras
from keras.models import Sequential, model_from_json                                            
from keras.layers.core import Dense, Dropout, Activation                       
from keras.layers.normalization import BatchNormalization
from keras.optimizers import SGD        
from keras.utils import np_utils
from keras.wrappers.scikit_learn import KerasRegressor
#sklearn
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# normal
import time
import os

Using TensorFlow backend.


Versions:

- keras: 2.0.8
- pandas: 0.23.4
- scikit-learn: 0.19.1
- tensorflow: 0.3.0
- theano: 1.0.2

*NOTE using tensorflow backend*

# Helper Functions

## Minkowski Distance

In [2]:
def minkowski_distance(x1, x2, y1, y2, p):
    return ((abs(x2 - x1) ** p) + (abs(y2 - y1)) ** p) ** (1 / p)

## Haversine Distance

In [3]:
# Radius of the earth in kilometers
R = 6378

def haversine_np(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees)

    All args must be of equal length.    
    
    source: https://stackoverflow.com/a/29546836

    """
    # Convert latitude and longitude to radians
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    # Find the differences
    dlon = lon2 - lon1
    dlat = lat2 - lat1

    # Apply the formula 
    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2
    # Calculate the angle (in radians)
    c = 2 * np.arcsin(np.sqrt(a))
    # Convert to kilometers
    km = R * c
    
    return km

## Date-Time

In [4]:
import re

def extract_dateinfo(df, date_col, drop=True, time=False, 
                     start_ref = pd.datetime(1900, 1, 1),
                     extra_attr = False):
    """
    Extract Date (and time) Information from a DataFrame
    Adapted from: https://github.com/fastai/fastai/blob/master/fastai/structured.py
    """
    df = df.copy()
    
    # Extract the field
    fld = df[date_col]
    
    # Check the time
    fld_dtype = fld.dtype
    if isinstance(fld_dtype, pd.core.dtypes.dtypes.DatetimeTZDtype):
        fld_dtype = np.datetime64

    # Convert to datetime if not already
    if not np.issubdtype(fld_dtype, np.datetime64):
        df[date_col] = fld = pd.to_datetime(fld, infer_datetime_format=True)
    

    # Prefix for new columns
    pre = re.sub('[Dd]ate', '', date_col)
    pre = re.sub('[Tt]ime', '', pre)
    
    # Basic attributes
    attr = ['Year', 'Month', 'Week', 'Day', 'Dayofweek', 'Dayofyear', 'Days_in_month', 'is_leap_year']
    
    # Additional attributes
    if extra_attr:
        attr = attr + ['Is_month_end', 'Is_month_start', 'Is_quarter_end', 
                       'Is_quarter_start', 'Is_year_end', 'Is_year_start']
    
    # If time is specified, extract time information
    if time: 
        attr = attr + ['Hour', 'Minute', 'Second']
        
    # Iterate through each attribute
    for n in attr: 
        df[pre + n] = getattr(fld.dt, n.lower())
        
    # Calculate days in year
    df[pre + 'Days_in_year'] = df[pre + 'is_leap_year'] + 365
        
    if time:
        # Add fractional time of day (0 - 1) units of day
        df[pre + 'frac_day'] = ((df[pre + 'Hour']) + (df[pre + 'Minute'] / 60) + (df[pre + 'Second'] / 60 / 60)) / 24
        
        # Add fractional time of week (0 - 1) units of week
        df[pre + 'frac_week'] = (df[pre + 'Dayofweek'] + df[pre + 'frac_day']) / 7
    
        # Add fractional time of month (0 - 1) units of month
        df[pre + 'frac_month'] = (df[pre + 'Day'] + (df[pre + 'frac_day'])) / (df[pre + 'Days_in_month'] +  1)
        
        # Add fractional time of year (0 - 1) units of year
        df[pre + 'frac_year'] = (df[pre + 'Dayofyear'] + df[pre + 'frac_day']) / (df[pre + 'Days_in_year'] + 1)
        
    # Add seconds since start of reference
    df[pre + 'Elapsed'] = (fld - start_ref).dt.total_seconds()
    
    if drop: 
        df = df.drop(date_col, axis=1)
        
    return df

# Load Data

In [5]:
if not os.path.exists('store.h5'):
# load the data
    data = pd.read_csv('input/train.csv', nrows=5_000_000,
        parse_dates = ['pickup_datetime']).drop(columns='key')
    # remove na
    ## I am not sure what na means
    data = data.dropna()
    with pd.HDFStore('store.h5') as store:
        store['df'] = data
    ## I am not sure what head does either
    data.head()
else:
    with pd.HDFStore('store.h5') as store:
        data = store['df']

# Sanitize Data

In [6]:
clean_data = data[data['fare_amount'].between(left=2.5, right=100)]
clean_data['fare-bin'] = pd.cut(clean_data['fare_amount'], bins = list(range(0, 50, 5))).astype(str)
# Uppermost bin
clean_data.loc[clean_data['fare-bin'] == 'nan', 'fare-bin'] = '[45+]'
# Adjust bin so the sorting is correct
clean_data.loc[clean_data['fare-bin'] == '(5, 10]', 'fare-bin'] = '(05, 10]'
clean_data = clean_data.loc[clean_data['passenger_count'] < 6]
# Remove latitude and longtiude outliers
clean_data = clean_data.loc[clean_data['pickup_latitude'].between(40, 42)]
clean_data = clean_data.loc[clean_data['pickup_longitude'].between(-75, -72)]
clean_data = clean_data.loc[clean_data['dropoff_latitude'].between(40, 42)]
clean_data = clean_data.loc[clean_data['dropoff_longitude'].between(-75, -72)]
clean_data = extract_dateinfo(clean_data, 'pickup_datetime', drop = False,
                              time = True, start_ref = data['pickup_datetime'].min())

/Users/harperic/miniconda3/envs/ml/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/harperic/miniconda3/envs/ml/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


# Feature Engineering

In [7]:
# Absolute difference in latitude and longitude
clean_data['abs_lat_diff'] = (clean_data['dropoff_latitude'] - clean_data['pickup_latitude']).abs()
clean_data['abs_lon_diff'] = (clean_data['dropoff_longitude'] - clean_data['pickup_longitude']).abs()

clean_data['manhattan'] = minkowski_distance(clean_data['pickup_longitude'], clean_data['dropoff_longitude'],
                                             clean_data['pickup_latitude'], clean_data['dropoff_latitude'], 1)

clean_data['euclidean'] = minkowski_distance(clean_data['pickup_longitude'], clean_data['dropoff_longitude'],
                                             clean_data['pickup_latitude'], clean_data['dropoff_latitude'], 2)



clean_data['haversine'] =  haversine_np(clean_data['pickup_longitude'], clean_data['pickup_latitude'],
                                        clean_data['dropoff_longitude'], clean_data['dropoff_latitude']) 

# Read in test data; create same features

In [8]:
test = pd.read_csv('input/test.csv', parse_dates = ['pickup_datetime'])
# Save the id for submission
test_id = list(test.pop('key'))

# Create absolute differences
test['abs_lat_diff'] = (test['dropoff_latitude'] - test['pickup_latitude']).abs()
test['abs_lon_diff'] = (test['dropoff_longitude'] - test['pickup_longitude']).abs()
test = extract_dateinfo(test, 'pickup_datetime', drop = False,
                        time = True, start_ref = data['pickup_datetime'].min())
test['manhattan'] = minkowski_distance(test['pickup_longitude'], test['dropoff_longitude'],
                                       test['pickup_latitude'], test['dropoff_latitude'], 1)

test['euclidean'] = minkowski_distance(test['pickup_longitude'], test['dropoff_longitude'],
                                       test['pickup_latitude'], test['dropoff_latitude'], 2)
test['haversine'] = haversine_np(test['pickup_longitude'], test['pickup_latitude'],
                                 test['dropoff_longitude'], test['dropoff_latitude'])

### Specify features

In [9]:
features = ['abs_lat_diff', 'abs_lon_diff', 'haversine', 'passenger_count',
            'pickup_latitude', 'pickup_longitude', 'dropoff_latitude', 'dropoff_longitude',
            'pickup_frac_day', 'pickup_frac_week', 'pickup_frac_year', 'pickup_Elapsed']

# set up Keras Dense ANN

In [10]:
def baseline_model():
    # create model
    model = Sequential()
    # need to get the number of all the features we're going to input
    model.add(BatchNormalization(input_shape=(len(features),)))
    model.add(Dense(len(features), activation="relu"))
    model.add(Dropout(0.25))
    model.add(Dense(1, kernel_initializer='normal'))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [11]:
seed = 7
np.random.seed(seed)
# evaluate model with standardized dataset
estimator = KerasRegressor(build_fn=baseline_model, epochs=100, batch_size=5, verbose=1)

### Split the training data for training/validation

In [12]:
X_train, X_valid, y_train, y_valid = train_test_split(clean_data, np.array(clean_data['fare_amount']), 
                                                      stratify = clean_data['fare-bin'],
                                                      random_state = seed, test_size = 1_000_000)

In [30]:
kfold = KFold(n_splits=10, random_state=seed)
X = np.asarray(X_train[features])
X = X[:10000]
y = np.asarray(y_train)
y = y[:10000]
results = cross_val_score(estimator, X, y, cv=kfold)
print("Results: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Epoch 1/100
9000/9000 [==============================] - 6s - loss: 78.4903     
Epoch 2/100
9000/9000 [==============================] - 6s - loss: 44.0329     
Epoch 3/100
9000/9000 [==============================] - 6s - loss: 40.9719     
Epoch 4/100
9000/9000 [==============================] - 8s - loss: 39.0070     
Epoch 5/100
9000/9000 [==============================] - 10s - loss: 38.9111    
Epoch 6/100
9000/9000 [==============================] - 12s - loss: 36.8690    
Epoch 7/100
9000/9000 [==============================] - 7s - loss: 36.2439     
Epoch 8/100
9000/9000 [==============================] - 6s - loss: 36.9328     
Epoch 9/100
9000/9000 [==============================] - 8s - loss: 36.6047     
Epoch 10/100
9000/9000 [==============================] - 7s - loss: 35.4293     
Epoch 11/100
9000/9000 [==============================] - 7s - loss: 34.6005     
Epoch 12/100
9000/9000 [==============================] - 8s - loss: 33.4081     
Epoch 13/100
9000/9000 [=

9000/9000 [==============================] - 6s - loss: 27.2666     
Epoch 100/100
9000/9000 [==============================] - 6s - loss: 84.8158     
Epoch 2/100
9000/9000 [==============================] - 6s - loss: 49.3499     
Epoch 3/100
9000/9000 [==============================] - 6s - loss: 42.6722     
Epoch 4/100
9000/9000 [==============================] - 6s - loss: 38.8618     
Epoch 5/100
9000/9000 [==============================] - 6s - loss: 37.4806     
Epoch 6/100
9000/9000 [==============================] - 6s - loss: 35.3077     
Epoch 7/100
9000/9000 [==============================] - 6s - loss: 34.4860     
Epoch 8/100
9000/9000 [==============================] - 6s - loss: 32.3977     
Epoch 9/100
9000/9000 [==============================] - 6s - loss: 32.8545     
Epoch 10/100
9000/9000 [==============================] - 6s - loss: 32.5014     
Epoch 11/100
9000/9000 [==============================] - 6s - loss: 32.3871     
Epoch 12/100
9000/9000 [============

9000/9000 [==============================] - 6s - loss: 27.1589     
Epoch 97/100
9000/9000 [==============================] - 6s - loss: 25.4624     
Epoch 98/100
9000/9000 [==============================] - 6s - loss: 26.2861     
Epoch 99/100
9000/9000 [==============================] - 6s - loss: 26.0698     
Epoch 100/100
9000/9000 [==============================] - 6s - loss: 75.8564     
Epoch 2/100
9000/9000 [==============================] - 6s - loss: 45.2713     
Epoch 3/100
9000/9000 [==============================] - 6s - loss: 42.9751     
Epoch 4/100
9000/9000 [==============================] - 6s - loss: 42.0421     
Epoch 5/100
9000/9000 [==============================] - 6s - loss: 40.8788     
Epoch 6/100
9000/9000 [==============================] - 6s - loss: 40.0444     
Epoch 7/100
9000/9000 [==============================] - 6s - loss: 38.2079     
Epoch 8/100
9000/9000 [==============================] - 6s - loss: 35.9178     
Epoch 9/100
9000/9000 [============

9000/9000 [==============================] - 7s - loss: 26.2584     
Epoch 94/100
9000/9000 [==============================] - 6s - loss: 26.8611     
Epoch 95/100
9000/9000 [==============================] - 5s - loss: 25.4803     
Epoch 96/100
9000/9000 [==============================] - 5s - loss: 26.1420     
Epoch 97/100
9000/9000 [==============================] - 6s - loss: 27.0878     
Epoch 98/100
9000/9000 [==============================] - 7s - loss: 26.7076     
Epoch 99/100
9000/9000 [==============================] - 6s - loss: 26.9544     
Epoch 100/100
9000/9000 [==============================] - 6s - loss: 78.0637     
Epoch 2/100
9000/9000 [==============================] - 7s - loss: 44.1166     
Epoch 3/100
9000/9000 [==============================] - 6s - loss: 39.6415     
Epoch 4/100
9000/9000 [==============================] - 6s - loss: 39.7133     
Epoch 5/100
9000/9000 [==============================] - 6s - loss: 38.2890     
Epoch 6/100
9000/9000 [=========

9000/9000 [==============================] - 5s - loss: 26.2500     
Epoch 92/100
9000/9000 [==============================] - 5s - loss: 26.0540     
Epoch 93/100
9000/9000 [==============================] - 5s - loss: 27.1705     
Epoch 94/100
9000/9000 [==============================] - 6s - loss: 26.3688     
Epoch 95/100
9000/9000 [==============================] - 5s - loss: 26.7357     
Epoch 96/100
9000/9000 [==============================] - 5s - loss: 26.2958     
Epoch 97/100
9000/9000 [==============================] - 5s - loss: 26.7406     
Epoch 98/100
9000/9000 [==============================] - 6s - loss: 26.4160     
Epoch 99/100
9000/9000 [==============================] - 6s - loss: 27.5470     
Epoch 100/100
9000/9000 [==============================] - 6s - loss: 80.8216     
Epoch 2/100
9000/9000 [==============================] - 6s - loss: 44.5781     
Epoch 3/100
9000/9000 [==============================] - 6s - loss: 41.0458     
Epoch 4/100
9000/9000 [=======

KeyboardInterrupt: 

In [26]:
print(X_train[features].shape)
print(y_train.shape)

(4779046, 12)
(4779046,)
